# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

# Read the CSV into DataFrame

In [13]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [14]:
# use label encoder for columns
label = LabelEncoder()

# Fit columns with the LabelEncoder
df['homeowner'] = label.fit_transform(df['homeowner'])
df['loan_status'] = label.fit_transform(df['loan_status'])

# Split the Data into Training and Testing

In [15]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [16]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [17]:
# Check the balance of our target values
y.head()

0    1
1    1
2    1
3    1
4    1
Name: loan_status, dtype: int32

In [18]:
y.value_counts()

1    75036
0     2500
Name: loan_status, dtype: int64

In [19]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=34)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [20]:
# Create the StandardScaler instance
scaler=StandardScaler()

In [21]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler=scaler.fit(X_train)

In [22]:
# Scale the training and testing data
X_train_scale=X_scaler.transform(X_train)
X_test_scale=X_scaler.transform(X_test)

# Simple Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9374163065514298

In [25]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  563,    77],
       [   91, 18653]], dtype=int64)

In [26]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.88      1.00      0.87      0.94      0.87       640
          1       1.00      1.00      0.88      1.00      0.94      0.89     18744

avg / total       0.99      0.99      0.88      0.99      0.94      0.88     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [28]:
# Resample the training data with the RandomOversampler
rand_sampler=RandomOverSampler(random_state=5)
X_resample, y_resample=rand_sampler.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resample)

Counter({1: 56292, 0: 56292})

In [29]:
# Train the Logistic Regression model using the resampled data
log_reg_model=LogisticRegression(solver='lbfgs', random_state=5)
log_reg_model.fit(X_resample, y_resample)

LogisticRegression(random_state=5)

In [30]:
# Calculated the balanced accuracy score
y_pred_log=log_reg_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_log)

0.9941541293213829

In [31]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred_log)

array([[  636,     4],
       [  102, 18642]], dtype=int64)

In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred_log))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.99      0.99      0.92      0.99      0.99       640
          1       1.00      0.99      0.99      1.00      0.99      0.99     18744

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [35]:
# Resample the training data with SMOTE
X_resample_SMOTE, y_resample_SMOTE=SMOTE(random_state=1).fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resample_SMOTE)

Counter({1: 56292, 0: 56292})

In [36]:
# Train the Logistic Regression model using the resampled data
log_smote=LogisticRegression(solver='lbfgs', random_state=1)
log_smote.fit(X_resample_SMOTE, y_resample_SMOTE)

LogisticRegression(random_state=1)

In [37]:
# Calculated the balanced accuracy score
y_pred_log_smote = log_smote.predict(X_test)
balanced_accuracy_score(y_test,y_pred_log_smote)

0.9941541293213829

In [38]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred_log_smote)

array([[  636,     4],
       [  102, 18642]], dtype=int64)

In [39]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred_log_smote))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.99      0.99      0.92      0.99      0.99       640
          1       1.00      0.99      0.99      1.00      0.99      0.99     18744

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [40]:
# Resample the data using the ClusterCentroids resampler
cluster_center=ClusterCentroids(random_state=3)
X_resampled_c, y_resampled_c = cluster_center.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled_c)

Counter({0: 1860, 1: 1860})

In [41]:
# Train the Logistic Regression model using the resampled data
log_cluster= LogisticRegression(solver='lbfgs', random_state=3)
log_cluster.fit(X_resampled_c, y_resampled_c)

LogisticRegression(random_state=3)

In [43]:
# Calculate the balanced accuracy score
y_pred_log_cluster=log_cluster.predict(X_test)
balanced_accuracy_score(y_test,y_pred_log_cluster)

0.975510830132309

In [45]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_log_cluster)

array([[  612,    28],
       [   98, 18646]], dtype=int64)

In [46]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred_log_cluster))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.96      0.99      0.91      0.98      0.95       640
          1       1.00      0.99      0.96      1.00      0.98      0.95     18744

avg / total       0.99      0.99      0.96      0.99      0.98      0.95     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [48]:
# Resample the training data with SMOTEENN
smoteenn_trainer=SMOTEENN(random_state=34)
X_resampled_smoteenn, y_resampled_smoteenn=smoteenn_trainer.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resample)

Counter({1: 56292, 0: 56292})

In [49]:
# Train the Logistic Regression model using the resampled data
smoteenn_model = LogisticRegression(solver='lbfgs', random_state=34)
smoteenn_model.fit(X_resampled_smoteenn, y_resampled_smoteenn)

LogisticRegression(random_state=34)

In [50]:
# Calculate the balanced accuracy score
y_pred_smoteenn_model = smoteenn_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smoteenn_model)

0.9940741037131882

In [51]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smoteenn_model)

array([[  636,     4],
       [  105, 18639]], dtype=int64)

In [52]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred_smoteenn_model))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.99      0.99      0.92      0.99      0.99       640
          1       1.00      0.99      0.99      1.00      0.99      0.99     18744

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   from the anaysis it looks like SMOTE and Naive scored very similar

2. Which model had the best recall score?

    all of the models we used showed the same .99

3. Which model had the best geometric mean score?

    it looked like Cluster Center, log regression were under the 1.0 of the other methods.
